# Neural network based regression - Densely connected network

In [1]:
import os
import sys

sys.path.append(os.path.join(os.path.abspath(''), ".."))

from torch import nn, optim

from nnbma.networks import DenselyConnected, PolynomialNetwork
from nnbma.layers import PolynomialExpansion
from nnbma.learning import LearningParameters, MaskedMSELoss

from helpers.preprocessing import get_names
from helpers.training import procedure
from helpers.results import save_results

In [2]:
filename = os.path.join(os.path.splitext(os.path.abspath(''))[0], "out-nn-regression-dense")

### Selected inputs (can be modified)

In [3]:
lines = None # If None, select all lines by default

In [4]:
inputs_names, outputs_names = get_names(lines=lines)
n_inputs, n_outputs = len(inputs_names), len(outputs_names)

### Architecture settings (can be modified)

In [5]:
n_layers = 12
growing_factor = 0.25
poly_degree = 3

In [6]:
## Architecture hyperparameters

n_expanded_inputs = PolynomialExpansion.expanded_features(poly_degree, n_inputs)

activation = nn.ELU()
batch_norm = False

## Network creation

subnetwork = DenselyConnected(
    n_expanded_inputs,
    n_outputs,
    n_layers,
    growing_factor,
    activation,
    batch_norm=batch_norm,
    outputs_names=outputs_names,
)

model = PolynomialNetwork(
    n_inputs,
    poly_degree,
    subnetwork,
    inputs_names=inputs_names,
    outputs_names=outputs_names,
    inputs_transformer=None, # Will be set in the procedure
    outputs_transformer=None, # Will be set in the procedure
)

layers_sizes
[34, 9, 11, 14, 17, 22, 27, 34, 42, 53, 66, 83, 5375]
412


### Training settings (can be modified)

In [7]:
# Epochs
epochs = 200

# Batch size
batch_size = 500

# Loss function
use_mask = True
loss = MaskedMSELoss() if use_mask else nn.MSELoss()

# Optimizer
learning_rate = 1e-3
optimizer = optim.Adam(model.parameters(), learning_rate)

# Scheduler
factor = 0.9
patience = 5
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, patience=patience, factor=factor, min_lr=learning_rate*1e-3
)

In [8]:
## Learning hyperparameters

learning_params = LearningParameters(loss, epochs, batch_size, optimizer, scheduler)

### Training procedure

In [9]:
results = procedure(
    outputs_names,
    model,
    learning_params,
    use_mask,
    verbose=True,
)


Number of inputs features: 4
Number of outputs features: 5,375
Number of rows: 19208

Number of parameters: 2,295,178 (9.18 MB)
Number of learnable parameters: 2,295,178 (9.18 MB)
Training initiated
PolynomialNetwork:
	input_features: 4
	order: 3
	subnetwork: DenselyConnected:
	input_features: 34
	output_features: 5375
	n_layers: 12
	growing_factor: 0.25
	activation: ELU(alpha=1.0)
	batch_norm: False
	inputs_names: None
	outputs_names: ['h2_v0_j2__v0_j0', 'h2_v0_j3__v0_j1', 'h2_v0_j4__v0_j2', 'h2_v0_j5__v0_j3', 'h2_v0_j6__v0_j4', 'h2_v0_j7__v0_j5', '...']
	inputs_transformer: None
	outputs_transformer: None
	device: cpu
	last_restrictable: True

	inputs_names: ['P', 'radm', 'Avmax', 'angle']
	outputs_names: ['h2_v0_j2__v0_j0', 'h2_v0_j3__v0_j1', 'h2_v0_j4__v0_j2', 'h2_v0_j5__v0_j3', 'h2_v0_j6__v0_j4', 'h2_v0_j7__v0_j5', '...']
	inputs_transformer: SequentialOperator: ["ColumnwiseOperator: ['log10', 'log10', 'log10', 'id']", 'Normalizer: NormTypes.MEAN0STD1']
	outputs_transformer: Oper

Epoch: 100%|██████████| 200/200 [23:06<00:00,  6.93s/it, train loss=0.0279, val loss=0.0237, train error=13930.89%, val error=42.19%]                                         


### Saving results

In [10]:
plot_profile = True

In [11]:
arch_name = f"layers_{n_layers}_factor_{growing_factor}_deg_{poly_degree}"

save_results(
    results,
    outputs_names,
    model,
    learning_params,
    use_mask,
    filename,
    architecture_name=arch_name,
    plot_profiles=plot_profile,
)


Number of inputs features: 4
Number of outputs features: 5,375
Number of rows: 19208

README saved
Learning figures saved
Architecture saved
Spreadsheets of errors saved


100%|██████████| 24510/24510 [00:00<00:00, 314198.80it/s]


Spreadsheet of worst estimations and their profiles saved
Spreadsheet of masked values saved
